In [3]:
# Estimate the parameters of Lotka-Volterra model.
# Parameters used for creating the data: r=[.1 .2]; a=[-1 .5;-1 -.5]; y[:,1]=[0.01,0.01] with 5% noise

using JuMP, Ipopt, Plots

# Data
t = linspace(0,24,101) # set of simulation times
t_exp = 1:10:101 # set of experimental times
# experimental observations
y_exp=[0.0106   0.0120  0.0170  0.0202  0.025  0.031  0.039  0.058  0.077  0.083  0.111
       0.0091   0.0147  0.0245  0.0358  0.045  0.081  0.092  0.133  0.153  0.160  0.168]

# create JuMP model object
m=Model(solver=IpoptSolver())

# define variables
@variable(m,  0 <= r[i in 1:2] <=10, start = 1) # growth rates
@variable(m, -3 <= a[i in 1:2, k in 1:2] <= 3, start =1) #  interaction parameters
@variable(m,  0 <= y[i in 1:2, k in 1:length(t)] <= 1, start = 1) # abundance

# define model
@NLconstraint(m, [i in 1:2, k in 1:length(t)-1],
  y[i,k+1] == y[i,k] + ( r[i] + sum( a[i,j] * y[j,k+1] for j in 1:2 )) * y[i,k+1] * (t[k+1]-t[k]))

# define objective
@objective(m, Min, sum((y[i,t_exp[k]]-y_exp[i,k])^2 for i in 1:2 for k in 1:length(t_exp)))

solve(m)

# Obtain the solution
r_sol = getvalue(r)
a_sol = getvalue(a)
y_sol = getvalue(y)
println("r_1 = $(r_sol[1]), a_11 = $(a_sol[1,1]), a_12 = $(a_sol[1,2])")
println("r_2 = $(r_sol[2]), a_21 = $(a_sol[2,1]), a_22 = $(a_sol[2,2])")

In [4]:
p=plot(t,y_sol',xlabel="Time (hr)", ylabel="Abundance", label=["Species 1" "Species 2"], 
                    color =[:blue :red], legend=:topleft)
plot!(p,t[t_exp],y_exp',label=["Species 1 (exp)" "Species 2 (exp)"],
                    seriestype=:scatter, color =[:blue :red])
savefig("param_est.pdf")

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     1200
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     3022

Total number of variables............................:      208
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      208
                     variables with only upper bounds:        0
Total number of equality constraints.................:      200
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  